# create a notebook to perform the RK method for multiple coupled variables

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    #set derivs
    
    #second order ODE is d^2y/dx^2=-y
    
    #broken into two first order ODE's:
    #dy/dx=z
    #dz/dx=-y
    #set y[0]=y and y[1]=z
    
    #now, declare an array
    y_derivs=np.zeros(2)
    
    #set dy/dx=z
    y_derivs[0]=y[1]
    
    #set dz/dx=-y
    y_derivs[1]=-1*y[0]
    
    #return array
    return y_derivs
    

## define a 6th order RK method

In [ ]:
def rk6_mv_core(dydx,xi,yi,nv,h):
    #declare arrays for each k
    k1=np.zeros(nv)
    k2=np.zeros(nv)
    k3=np.zeros(nv)
    k4=np.zeros(nv)
    k5=np.zeros(nv)
    k6=np.zeros(nv)
    
    #define x each k according to Cash-Karp method
    x_1=xi
    x_2=xi+0.2*h
    x_3=xi+(3*h)/10
    x_4=xi+(3*h)/5
    x_5=xi+h
    x_6=xi+(7*h)/8
    
    #declare temp y arrays since the values of y will be updatted as this code progresses
    y_temp=np.zeros(nv)
    
    #get k1 values
    y_derivs=dydx(x_1,yi)
    k1[:]=h*y_derivs[:]
    
    #get k2 values
    y_temp[:]=yi[:]+0.2*k1[:]
    y_derivs=dydx(x_2,y_temp)
    k2[:]=h*y_derivs[:]
    
    #get k3 values
    y_temp[:]=yi[:]+(3/40)*k1[:]+(9/40)*k2[:]
    y_derivs=dydx(x_3,y_temp)
    k3[:]=h*y_derivs[:]
    
    #get k4 values
    y_temp[:]=yi[:]+(3/10)*k1[:]+(-9/10)*k2[:]+(6/5)*k3[:]
    y_derivs=dydx(x_4,y_temp)
    k4[:]=h*y_derivs[:]
    
    #get k5 values
    y_temp[:]=yi[:]+(-11/54)*k1[:]+(5/2)*k2[:]+(-70/27)*k3[:]+(35/27)*k4[:]
    y_derivs=dydx(x_5,y_temp)
    k5[:]=h*y_derivs[:]
    
    #get k6 values
    y_temp[:]=yi[:]+(1631/55296)*k1[:]+(175/512)*k2[:]+(575/13824)*k3[:]+(44275/110592)*k4[:]+(353/4096)*k5[:]
    y_derivs=dydx(x_6,y_temp)
    k6[:]=h*y_derivs[:]
    
    #advance y by the full step, h for both 5th and 6th order approximations
    y_5=yi[:]+(2825/27648)*k1[:]+(18575/48384)*k3[:]+(13525/55296)*k4[:]+(277/14336)*k5[:]+(1/4)*k6[:]
    y_6=yi[:]+(37/378)*k1[:]+(250/621)*k3[:]+(125/594)*k4[:]+(512/1771)*k6[:]
    
    return y_5, y_6

## define an adaptive step driver for RK6

In [ ]:
def rk6_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max number of iterations, imax
    imax=10000
    
    #set an iteration value
    i=1
    
    #create an error
    delta=np.full(nv,2*tol)
    
    #assign the step to a variable
    h_step=h
    
    #adjust the step if too large
    while(delta.max()/tol>1.0):
        
        #approximate the error of our method by the difference in approximations of the 5th and 6th order
        y_2, y_1 = rk6_mv_core(dydx,x_i,y_i,nv,h_step)
        
        
        #now, we determine the error
        delta=np.fabs(y_2-y_1)
        
        #if our error is too large, we take a smaller step size
        if(delta.max()/tol>1.0):
            
            #we adjust the step by making it smaller using our safety method
            h_step*=SAFETY*(delta.max()/tol)**(-0.25)
            
        #now we check our iteration has not gone over imax
        if(i>=imax):
            print("Too many iterations in rk6_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
    
    #for the next try, we take a bigger step
    h_new=np.fmin(h_step*(delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we just took
    return y_2, h_new, h_step

## define a wrapper for RK6

In [ ]:
def rk6_mv(dydx,a,b,y_a,tol):
    #dydx is the deriv function
    #a is the lower bound, starting point
    #b is the upper bound, ending point
    #y_a is the initial condition/boundary condition
    #tol is the tolerance of error we accept when integrating y
    
    #define a starting step
    xi=a
    yi=y_a.copy()
    
    #we make our first step very small
    h=1.0e-4*(b-a)
    
    #set a max number of iterations like before
    imax=10000
    
    #set an iteration value like before
    i=0
    
    #set the number of coupled ode's to the size of y_a. Since there are two values of y_a, there will be two ode's
    nv=len(y_a)
    
    #set up our initial conditions
    #x need not be a 2d array because we only have one set of x values for both ode's
    #y will need to be a 2d array since we have two ode's
    x=np.full(1,a)
    y=np.full((1,nv),y_a)
    
    #set flag
    flag=1
    
    #we create a loop until we end up at the right side of our domain, namely b
    while(flag):
        #calculate y_i+1
        yi_new, h_new, h_step = rk6_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h=h_new
        
        #prevent overshooting of the right side boundary, b
        if(xi+h_step>b):
            
            #we take a smaller step
            h=b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk6_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #code break
            flag=0
        
        #we then update values
        xi+=h_step
        yi[:]=yi_new[:]
        
        #add the step to the arrays
        x=np.append(x,xi)
        y_new=np.zeros((len(x),nv))
        y_new[0:len(x)-1,:]=y
        y_new[-1,:]=yi[:]
        del y
        y=y_new
        
        #if we go through too many iterations
        if(i>=imax):            
            print("Max iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        #iterate
        i+=1
        
        #output some info
        s="i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break code if new xi == b
        if(xi==b):
            flag==0
            
    #return our answer
    return x,y     

## perform integration, finally

In [ ]:
a=0
b=2*np.pi

y_0=np.zeros(2)
y_0[0]=0
y_0[1]=1
nv=2

tolerance=1.0e-6

#perform the integration
x,y = rk6_mv(dydx,a,b,y_0,tolerance)

## plot result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dy/dx (x)')
plt.xlim([0,2*np.pi])

#plot sin(x) and cos(x) curves
xx=np.linspace(0,2*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y,dy/dx')
plt.legend(frameon=False)

## Plot the error

In [ ]:
sine=np.sin(x)
cosine=np.cos(x)

y_error=(y[:,0]-sine)
dydx_error=(y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) error")
plt.plot(x, dydx_error, label="dy/dx (x) error")
plt.xlim([0,2*np.pi])
plt.legend(frameon=False)